<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/download_osm_street_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing and loading dependencies

In [1]:
!pip install -q osmnx
!pip install -q h3
!pip install -q pandana

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import shapely
from lxml import etree
import h3
import pandana as pdna
from glob import glob
import os

## Downloading the osmnx network using bbox

To draw the bounding box (bbox) you can use this [tool](https://bboxfinder.com/)
and this [Bounding Box Tool](https://boundingbox.klokantech.com/)

- North = ymax
- South = ymin
- East = xmax
- West = xmin

In [3]:
from pandana.loaders import osm

In [11]:
## Note: the coordinates are in West, South, East, North format

bboxs = {
    'Mumbai': (72.7712628039,18.8895552464,73.0912396106,19.322318006),
    'Hyderabad': (78.2288393192,17.1952635128,78.692325037,17.6244114807),
    'Bangalore': (77.4480254838,12.815481916,77.7549560258,13.1026335949),
    'Chennai': (80.0337033915,12.8678089117,80.335827415,13.2370438819),
    'Kolkata': (88.2431406976,22.4883995072,88.4944529534,22.7071046611),
    'Ahmedabad': (72.4617185823,22.9179551449,72.699297928,23.1470216925)
}

In [17]:
%%time
city = 'Bangalore'
W,S,E,N = bboxs[city]
ox_network = ox.graph.graph_from_bbox(bbox=(N,S,E,W), network_type='all', simplify=True, retain_all=False, truncate_by_edge=False, clean_periphery=None, custom_filter=None)

CPU times: user 4min 20s, sys: 3.93 s, total: 4min 24s
Wall time: 4min 38s


How to load pandana network from OSM:
https://udst.github.io/pandana/loaders.html

In [18]:
%%time
## Convert the osmx.Graph Object to geopandas
nodes, edges = ox.convert.graph_to_gdfs(ox_network, nodes=True, edges=True) #node_geometry=True, fill_edge_geometry=True
edges = edges.reset_index()

CPU times: user 1min 17s, sys: 1.94 s, total: 1min 19s
Wall time: 1min 20s


## Export the networks

In [19]:
%%time
network = pdna.Network(nodes["x"], nodes["y"], edges["u"], edges["v"],edges[["length"]])

CPU times: user 48.7 s, sys: 406 ms, total: 49.2 s
Wall time: 37.9 s


In [20]:
network.save_hdf5(f'{city}_network.h5')

## Read the exported network data

In [21]:
%%time
network = pdna.Network.from_hdf5(f'{city}_network.h5')

CPU times: user 46 s, sys: 398 ms, total: 46.4 s
Wall time: 35 s


## Read KMZ files

The metro stations and lines are in KML format. We need to convert it to a gdf for further analysis

In [ ]:
!wget https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/data/delhi-metro-lines.kml -q -O delhi-metro-lines.kml
!wget https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/data/Delhi%20Metro%20Phase%201-4%20Route%20Map%20-%20By%20Line.kml -q -O delhi-metro-stations.kml

In [ ]:
# Load the KML file
tree = etree.parse('delhi-metro-stations.kml')
tree

Read the exported

In [ ]:
data = []
lines = []
folders = tree.findall('.//{http://www.opengis.net/kml/2.2}Folder')
for folder in folders:
    placemarks = tree.findall('.//{http://www.opengis.net/kml/2.2}Placemark')
    for placemark in placemarks:
        point_element = placemark.find('.//{http://www.opengis.net/kml/2.2}Point')
        line_element = placemark.find('.//{http://www.opengis.net/kml/2.2}LineString')

        name = placemark.find('.//{http://www.opengis.net/kml/2.2}name').text.replace("\xa0", " ").strip()
        phase = folder.find('.//{http://www.opengis.net/kml/2.2}name').text.replace("\xa0", " ").strip()

        if point_element:
            coordinates = eval(point_element.find('.//{http://www.opengis.net/kml/2.2}coordinates').text)
            x,y = coordinates[:2]

            geometry = shapely.geometry.Point(x,y)
            data.append([name, phase, geometry])

        if line_element:
            line_coordinates = line_element.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
            geometry = shapely.geometry.LineString([eval(x)[:2] for x in line_coordinates.strip().split(" ")])
            lines.append([name, phase, geometry])


def route_name(name):
    if "-" in name:
        route_name = name.split("-")[0].strip()
    else:
        route_name = name.split(" ")[0].strip()
    return route_name

color_data = {
    'Yellow': (255, 255, 0),
    'Red': (255, 0, 0),
    'Blue': (0, 0, 255),
    'Green': (0, 128, 0),
    'Violet': (148, 0, 211),
    'Orange': (255, 165, 0),
    'Green line': (0, 128, 0),
    'Pink': (255, 192, 203),
    'Grey': (128, 128, 128),
    'Magenta': (255, 0, 255),
    'Silver': (192, 192, 192),
    "Golden Line's Spur": (218, 165, 32),
    'New Line': (0,0,0)
}

## Reference
1. Uploading large files in github: https://git-lfs.com/

In [ ]:
metro_stations = gpd.GeoDataFrame(data, geometry="geometry", columns=['Name', "Phase", 'geometry'], crs='EPSG:4326')
metro_stations.head()

,Name,Phase,geometry
0,Yamuna Bank Depot,Phase 1 - 65.1 km,POINT (77.26441 28.62344)
1,Najafgarh Depot,Phase 1 - 65.1 km,POINT (77.00586 28.61156)
2,Sultanpur Depot,Phase 1 - 65.1 km,POINT (77.15345 28.49253)
3,Ajronda Depot,Phase 1 - 65.1 km,POINT (77.30950 28.39947)
4,Sector 21 Depot (Airport Exp),Phase 1 - 65.1 km,POINT (77.06606 28.56077)


### Colormapping the metro lines

In [ ]:
metro_lines['Name'].apply(route_name)

0                  Yellow
1                     Red
2                    Blue
3                     Red
4                  Yellow
              ...        
315                   Red
316                  Blue
317    Golden Line's Spur
318                  Blue
319              New Line
Name: Name, Length: 320, dtype: object

In [ ]:
metro_lines = gpd.GeoDataFrame(lines, geometry="geometry", columns=['Name', "Phase", 'geometry'], crs='EPSG:4326')
metro_lines['color'] = metro_lines['Name'].apply(route_name).apply(lambda x: color_data[x])
metro_lines['path'] = metro_lines['geometry'].apply(lambda x: list(x.coords))
metro_lines['display_text'] = metro_lines[['Name', 'Phase']].apply(lambda x: f"Metro Route: {x[0]}<br>Phase: {x[1]}", axis=1)
metro_lines.sample(5)

,Name,Phase,geometry,color,path,display_text
34,Blue (32.1 km),Phase 2 - 124.93 km,"LINESTRING (77.25320 28.62020, 77.25200 28.620...","(0, 0, 255)","[(77.2531986, 28.6202029), (77.251997, 28.6201...",Metro Route: Blue (32.1 km)<br>Phase: Phase 2 ...
201,Green - Kirti Nagar (3.31 km),Phase 1 & 2 Stations,"LINESTRING (77.15703 28.67131, 77.15868 28.670...","(0, 128, 0)","[(77.1570307, 28.671311), (77.1586803, 28.6709...",Metro Route: Green - Kirti Nagar (3.31 km)<br>...
269,Red - Ghaziabad (9.41 km),Phase 4 Stations,"LINESTRING (77.32251 28.67590, 77.33218 28.676...","(255, 0, 0)","[(77.3225069, 28.6759045), (77.332184, 28.6765...",Metro Route: Red - Ghaziabad (9.41 km)<br>Phas...
193,Red (22 km),Phase 1 & 2 Stations,"LINESTRING (77.10234 28.72449, 77.11116 28.717...","(255, 0, 0)","[(77.1023404, 28.7244921), (77.1111629, 28.717...",Metro Route: Red (22 km)<br>Phase: Phase 1 & 2...
155,Red - Nathupur (4.86 km),Phase 4 (Lines Pending Approval),"LINESTRING (77.09785 28.85452, 77.09785 28.856...","(255, 0, 0)","[(77.097845, 28.854522), (77.0978504, 28.85626...",Metro Route: Red - Nathupur (4.86 km)<br>Phase...


## Calculate the accessibility

In [ ]:
%%time
# initialize each amenity category with the locations (lon/lat coordinates)

max_distance=25000 ## in meters
num_pois = 1

category = 'metro_stations'
network.set_pois(category=category, maxdist = max_distance, maxitems=num_pois, x_col=metro_stations['geometry'].x, y_col=metro_stations['geometry'].y)
accessibility = network.nearest_pois(distance = max_distance, category=category, num_pois=num_pois)